In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing the Required Library
import pandas as  pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Dense,Activation,Conv2D,MaxPooling2D,Flatten, BatchNormalization, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
x_test = pd.read_csv("../input/digit-recognizer/test.csv")
train.head()

In [ ]:
# taking the independent feature and the dependent feature in xtrain and ytrain of the traning data
x_train = train.drop(columns = 'label')
y_train = train['label']

In [ ]:
x_test.shape,x_train.shape,y_train.shape

In [ ]:
sns.countplot(y_train)

In [ ]:
#data processing
x_train = x_train.values.reshape(-1, int(np.sqrt(784)), int(np.sqrt(784)), 1)/255.0
x_test =  x_test.values.reshape(-1, int(np.sqrt(784)), int(np.sqrt(784)), 1)/255.0

In [ ]:
x_test.shape,x_train.shape,y_train.shape

In [ ]:
rows = 5
cols = 6
counter = 0
fig = plt.figure(figsize=(15,7))
for i in range(1, rows*cols+1):
    fig.add_subplot(rows, cols, i)
    plt.imshow(np.squeeze(x_train[counter + i-1]), cmap='gray')
    plt.title(y_train[counter + i-1], fontsize=16)
    plt.axis(False)
    fig.add_subplot
counter += rows*cols

In [ ]:
y_train = to_categorical(y_train, num_classes=10)

In [ ]:
x_test.shape,x_train.shape,y_train.shape

In [ ]:
x_train_trim ,x_valid , y_train_trim, y_valid = train_test_split(x_train, y_train, test_size= 0.1 ,random_state = 1455)
print(f'Training Set size: {x_train_trim.shape[0]}')
print(f'Validation Set size: {x_valid.shape[0]}')

In [ ]:
model = Sequential()#69
input_shape = (28,28,1)
model.add(Conv2D(32, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image("model.png")

In [ ]:
#Optimizer & model compile
#optimizer_rmsprop = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
adam = Adam(lr = .00001)
model.compile(optimizer=adam, 
              loss = 'categorical_crossentropy',  
              metrics = ['accuracy'])

In [ ]:
# With data augmentation to prevent overfitting (accuracy 0.99286)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=23,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train_trim)

In [ ]:
def build_lrfn(lr_start=1e-4, lr_max=1e-3, 
               lr_min=0, lr_rampup_epochs=16, 
               lr_sustain_epochs=0, lr_exp_decay=.8):

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

lrfn = build_lrfn()
lr_schedule = LearningRateScheduler(lrfn, verbose=True)

#Usually monitor='val_accuracy' should be tracked here. Since the training set is smaller let keep it limited to accuracy
checkpoint = ModelCheckpoint(
    filepath='best_weights.hdf5',
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
train_history = model.fit_generator(datagen.flow(x_train_trim, y_train_trim, 
                                                 batch_size=512),
                                    epochs = 50,
                                    validation_data = (x_valid,y_valid),
                                    verbose = 2, 
                                    steps_per_epoch=x_train_trim.shape[0] //512,
                                    callbacks=[lr_schedule,checkpoint])

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0001.
Epoch 1/50
147/147 - 9s - loss: 0.5003 - accuracy: 0.8416 - val_loss: 2.4651 - val_accuracy: 0.1012 - lr: 1.0000e-04

Epoch 00002: LearningRateScheduler reducing learning rate to 0.00015625.
Epoch 2/50
147/147 - 9s - loss: 0.1386 - accuracy: 0.9583 - val_loss: 1.9740 - val_accuracy: 0.2471 - lr: 1.5625e-04

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00021250000000000002.
Epoch 3/50
147/147 - 9s - loss: 0.0981 - accuracy: 0.9701 - val_loss: 0.4611 - val_accuracy: 0.8886 - lr: 2.1250e-04

Epoch 00004: LearningRateScheduler reducing learning rate to 0.00026875.
Epoch 4/50
147/147 - 9s - loss: 0.0852 - accuracy: 0.9740 - val_loss: 0.0495 - val_accuracy: 0.9876 - lr: 2.6875e-04

Epoch 00005: LearningRateScheduler reducing learning rate to 0.000325.
Epoch 5/50
147/147 - 10s - loss: 0.0680 - accuracy: 0.9794 - val_loss: 0.0466 - val_accuracy: 0.9874 - lr: 3.2500e-04

Epoch 00006: LearningRateScheduler reducing learning rate to 0.00038124999999999997.
Epoch 6/50
147/147 - 9s - loss: 0.0651 - accuracy: 0.9806 - val_loss: 0.0603 - val_accuracy: 0.9824 - lr: 3.8125e-04

Epoch 00007: LearningRateScheduler reducing learning rate to 0.0004375.
Epoch 7/50
147/147 - 9s - loss: 0.0532 - accuracy: 0.9840 - val_loss: 0.0416 - val_accuracy: 0.9876 - lr: 4.3750e-04

Epoch 00008: LearningRateScheduler reducing learning rate to 0.00049375.
Epoch 8/50
147/147 - 9s - loss: 0.0477 - accuracy: 0.9859 - val_loss: 0.0410 - val_accuracy: 0.9883 - lr: 4.9375e-04


Epoch 00009: LearningRateScheduler reducing learning rate to 0.00055.
Epoch 9/50
147/147 - 9s - loss: 0.0470 - accuracy: 0.9858 - val_loss: 0.1555 - val_accuracy: 0.9540 - lr: 5.5000e-04

Epoch 00010: LearningRateScheduler reducing learning rate to 0.00060625.
Epoch 10/50
147/147 - 9s - loss: 0.0484 - accuracy: 0.9851 - val_loss: 0.0368 - val_accuracy: 0.9912 - lr: 6.0625e-04

Epoch 00011: LearningRateScheduler reducing learning rate to 0.0006625.
Epoch 11/50
147/147 - 9s - loss: 0.0425 - accuracy: 0.9882 - val_loss: 0.0506 - val_accuracy: 0.9864 - lr: 6.6250e-04

Epoch 00012: LearningRateScheduler reducing learning rate to 0.00071875.
Epoch 12/50
147/147 - 9s - loss: 0.0372 - accuracy: 0.9890 - val_loss: 0.0984 - val_accuracy: 0.9750 - lr: 7.1875e-04

Epoch 00013: LearningRateScheduler reducing learning rate to 0.0007750000000000001.
Epoch 13/50
147/147 - 9s - loss: 0.0402 - accuracy: 0.9879 - val_loss: 0.1463 - val_accuracy: 0.9598 - lr: 7.7500e-04

Epoch 00014: LearningRateScheduler reducing learning rate to 0.0008312500000000001.
Epoch 14/50
147/147 - 9s - loss: 0.0336 - accuracy: 0.9897 - val_loss: 0.0246 - val_accuracy: 0.9921 - lr: 8.3125e-04

Epoch 00015: LearningRateScheduler reducing learning rate to 0.0008875.
Epoch 15/50
147/147 - 9s - loss: 0.0369 - accuracy: 0.9891 - val_loss: 0.0789 - val_accuracy: 0.9790 - lr: 8.8750e-04

Epoch 00016: LearningRateScheduler reducing learning rate to 0.00094375.
Epoch 16/50
147/147 - 9s - loss: 0.0340 - accuracy: 0.9901 - val_loss: 0.0334 - val_accuracy: 0.9907 - lr: 9.4375e-04

Epoch 00017: LearningRateScheduler reducing learning rate to 0.001.
Epoch 17/50
147/147 - 9s - loss: 0.0308 - accuracy: 0.9905 - val_loss: 0.0454 - val_accuracy: 0.9867 - lr: 0.0010

Epoch 00018: LearningRateScheduler reducing learning rate to 0.0008.
Epoch 18/50
147/147 - 9s - loss: 0.0268 - accuracy: 0.9919 - val_loss: 0.0313 - val_accuracy: 0.9895 - lr: 8.0000e-04

Epoch 00019: LearningRateScheduler reducing learning rate to 0.0006400000000000002.
Epoch 19/50
147/147 - 9s - loss: 0.0201 - accuracy: 0.9940 - val_loss: 0.0332 - val_accuracy: 0.9929 - lr: 6.4000e-04

Epoch 00020: LearningRateScheduler reducing learning rate to 0.0005120000000000001.
Epoch 20/50
147/147 - 9s - loss: 0.0167 - accuracy: 0.9950 - val_loss: 0.0324 - val_accuracy: 0.9898 - lr: 5.1200e-04

Epoch 00021: LearningRateScheduler reducing learning rate to 0.0004096000000000001.
Epoch 21/50
147/147 - 9s - loss: 0.0130 - accuracy: 0.9961 - val_loss: 0.0272 - val_accuracy: 0.9921 - lr: 4.0960e-04

Epoch 00022: LearningRateScheduler reducing learning rate to 0.0003276800000000001.
Epoch 22/50
147/147 - 9s - loss: 0.0112 - accuracy: 0.9968 - val_loss: 0.0203 - val_accuracy: 0.9950 - lr: 3.2768e-04


Epoch 00023: LearningRateScheduler reducing learning rate to 0.0002621440000000001.
Epoch 23/50
147/147 - 9s - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.0226 - val_accuracy: 0.9938 - lr: 2.6214e-04

Epoch 00024: LearningRateScheduler reducing learning rate to 0.0002097152000000001.
Epoch 24/50
147/147 - 9s - loss: 0.0083 - accuracy: 0.9972 - val_loss: 0.0181 - val_accuracy: 0.9945 - lr: 2.0972e-04

Epoch 00025: LearningRateScheduler reducing learning rate to 0.0001677721600000001.
Epoch 25/50
147/147 - 9s - loss: 0.0090 - accuracy: 0.9974 - val_loss: 0.0201 - val_accuracy: 0.9940 - lr: 1.6777e-04

Epoch 00026: LearningRateScheduler reducing learning rate to 0.00013421772800000008.
Epoch 26/50
147/147 - 9s - loss: 0.0061 - accuracy: 0.9980 - val_loss: 0.0179 - val_accuracy: 0.9948 - lr: 1.3422e-04

Epoch 00027: LearningRateScheduler reducing learning rate to 0.00010737418240000006.
Epoch 27/50
147/147 - 9s - loss: 0.0065 - accuracy: 0.9981 - val_loss: 0.0219 - val_accuracy: 0.9940 - lr: 1.0737e-04

Epoch 00028: LearningRateScheduler reducing learning rate to 8.589934592000005e-05.
Epoch 28/50
147/147 - 9s - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.0209 - val_accuracy: 0.9950 - lr: 8.5899e-05

Epoch 00029: LearningRateScheduler reducing learning rate to 6.871947673600005e-05.
Epoch 29/50
147/147 - 9s - loss: 0.0059 - accuracy: 0.9980 - val_loss: 0.0185 - val_accuracy: 0.9955 - lr: 6.8719e-05

Epoch 00030: LearningRateScheduler reducing learning rate to 5.497558138880004e-05.
Epoch 30/50
147/147 - 9s - loss: 0.0047 - accuracy: 0.9985 - val_loss: 0.0179 - val_accuracy: 0.9952 - lr: 5.4976e-05

Epoch 00031: LearningRateScheduler reducing learning rate to 4.398046511104004e-05.
Epoch 31/50
147/147 - 9s - loss: 0.0048 - accuracy: 0.9985 - val_loss: 0.0202 - val_accuracy: 0.9952 - lr: 4.3980e-05

Epoch 00032: LearningRateScheduler reducing learning rate to 3.518437208883203e-05.
Epoch 32/50
147/147 - 9s - loss: 0.0041 - accuracy: 0.9989 - val_loss: 0.0199 - val_accuracy: 0.9952 - lr: 3.5184e-05

Epoch 00033: LearningRateScheduler reducing learning rate to 2.8147497671065623e-05.
Epoch 33/50
147/147 - 9s - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.0191 - val_accuracy: 0.9955 - lr: 2.8147e-05

Epoch 00034: LearningRateScheduler reducing learning rate to 2.2517998136852502e-05.
Epoch 34/50
147/147 - 9s - loss: 0.0042 - accuracy: 0.9985 - val_loss: 0.0192 - val_accuracy: 0.9955 - lr: 2.2518e-05

Epoch 00035: LearningRateScheduler reducing learning rate to 1.8014398509482003e-05.
Epoch 35/50
147/147 - 9s - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0192 - val_accuracy: 0.9955 - lr: 1.8014e-05

Epoch 00036: LearningRateScheduler reducing learning rate to 1.4411518807585603e-05.
Epoch 36/50
147/147 - 9s - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0193 - val_accuracy: 0.9957 - lr: 1.4412e-05

Epoch 00037: LearningRateScheduler reducing learning rate to 1.1529215046068483e-05.
Epoch 37/50
147/147 - 9s - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.0186 - val_accuracy: 0.9957 - lr: 1.1529e-05

Epoch 00038: LearningRateScheduler reducing learning rate to 9.223372036854787e-06.
Epoch 38/50
147/147 - 9s - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.0188 - val_accuracy: 0.9955 - lr: 9.2234e-06

Epoch 00039: LearningRateScheduler reducing learning rate to 7.37869762948383e-06.
Epoch 39/50
147/147 - 9s - loss: 0.0038 - accuracy: 0.9987 - val_loss: 0.0185 - val_accuracy: 0.9955 - lr: 7.3787e-06

Epoch 00040: LearningRateScheduler reducing learning rate to 5.902958103587064e-06.
Epoch 40/50
147/147 - 9s - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0187 - val_accuracy: 0.9957 - lr: 5.9030e-06

Epoch 00041: LearningRateScheduler reducing learning rate to 4.722366482869652e-06.
Epoch 41/50
147/147 - 9s - loss: 0.0038 - accuracy: 0.9988 - val_loss: 0.0185 - val_accuracy: 0.9957 - lr: 4.7224e-06

Epoch 00042: LearningRateScheduler reducing learning rate to 3.7778931862957216e-06.
Epoch 42/50
147/147 - 9s - loss: 0.0038 - accuracy: 0.9989 - val_loss: 0.0185 - val_accuracy: 0.9957 - lr: 3.7779e-06

Epoch 00043: LearningRateScheduler reducing learning rate to 3.0223145490365774e-06.
Epoch 43/50
147/147 - 9s - loss: 0.0034 - accuracy: 0.9989 - val_loss: 0.0186 - val_accuracy: 0.9957 - lr: 3.0223e-06


Epoch 00044: LearningRateScheduler reducing learning rate to 2.417851639229262e-06.
Epoch 44/50
73/73 - 9s - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.0237 - val_accuracy: 0.9960 - lr: 2.4179e-06

Epoch 00045: LearningRateScheduler reducing learning rate to 1.93428131138341e-06.
Epoch 45/50
73/73 - 9s - loss: 0.0020 - accuracy: 0.9993 - val_loss: 0.0237 - val_accuracy: 0.9960 - lr: 1.9343e-06

Epoch 00046: LearningRateScheduler reducing learning rate to 1.547425049106728e-06.
Epoch 46/50
73/73 - 9s - loss: 0.0022 - accuracy: 0.9994 - val_loss: 0.0238 - val_accuracy: 0.9960 - lr: 1.5474e-06

Epoch 00047: LearningRateScheduler reducing learning rate to 1.2379400392853823e-06.
Epoch 47/50
73/73 - 9s - loss: 0.0024 - accuracy: 0.9992 - val_loss: 0.0238 - val_accuracy: 0.9960 - lr: 1.2379e-06

Epoch 00048: LearningRateScheduler reducing learning rate to 9.903520314283058e-07.
Epoch 48/50
73/73 - 9s - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0238 - val_accuracy: 0.9960 - lr: 9.9035e-07

Epoch 00049: LearningRateScheduler reducing learning rate to 7.922816251426449e-07.
Epoch 49/50
73/73 - 9s - loss: 0.0024 - accuracy: 0.9992 - val_loss: 0.0238 - val_accuracy: 0.9960 - lr: 7.9228e-07

Epoch 00050: LearningRateScheduler reducing learning rate to 6.338253001141158e-07.
Epoch 50/50
73/73 - 9s - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0237 - val_accuracy: 0.9960 - lr: 6.3383e-07

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
my_submission = pd.DataFrame({'ImageId': list(range(1, len(y_pred)+1)), 'Label': y_pred})
my_submission.to_csv('submission.csv', index=False)